### It is necessary to change path of the database located in the github folder /data

In [1]:
import numpy as np
import pandas as pd
import os


column_names = ['patient_number', 'age', 'sex', 'adult_BMI', 'child_weight', 'child_height']
dataset = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', sep=" ", header=None, names =column_names)
dataset['sex'] = dataset['sex'].replace({'F': 1, 'M': 0})


column_names = ['Beginning of respiratory cycle(s)','End of respiratory cycle(s)','Crackles','Wheezes']
patient_dataset = {}

for file in os.listdir("../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files"):
        if file.endswith(".txt"):
            patient_number = file.split("_")[0]
            data = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'+file, sep="\t", header=None, names = column_names)
            for crackles in data['Crackles']:
                if crackles == 1:
                    if(patient_dataset.get(patient_number+"_crackles") != None):
                        patient_dataset[patient_number+"_crackles"] += 1
                    else:
                        patient_dataset[patient_number+"_crackles"] = 1;
            for wheezes in data['Wheezes']:
                if wheezes == 1:
                    if(patient_dataset.get(patient_number+"_wheezes") != None):
                        patient_dataset[patient_number+"_wheezes"] += 1
                    else:
                        patient_dataset[patient_number+"_wheezes"] = 1;
            
final_patient_dataset = []

new_data = {#'patient_number': [pd.to_numeric(101)], 
               'n_wheezes': [patient_dataset.get(str(101) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(101) + ("_crackles"))]}
new_data = pd.DataFrame(new_data)
final_strings_for_dataset = pd.DataFrame(new_data)


for patient_number in range (102, 227):  
    
    new_data = {#'patient_number': [pd.to_numeric(patient_number)], 
               'n_wheezes': [patient_dataset.get(str(patient_number) + ("_wheezes"))],
               'n_crackles': [patient_dataset.get(str(patient_number) + ("_crackles"))]}
    new_data = pd.DataFrame(new_data)
    final_strings_for_dataset = final_strings_for_dataset.append([new_data],  ignore_index=True)


final_strings_for_dataset.loc[~final_strings_for_dataset.index.duplicated(keep='first')]
dataset.loc[~dataset.index.duplicated(keep='first')]

dataset = pd.concat([dataset, final_strings_for_dataset], axis=1)
dataset = dataset.replace(np.nan, '0')


#I am not sure of this, http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html
Y = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv', sep=",", header=None, names=['patient_number', 'diagnosis'])
Y = Y['diagnosis']

'''
['URTI' 'Healthy' 'Asthma' 'COPD' 'LRTI' 'Bronchiectasis' 'Pneumonia' 'Bronchiolitis']
'''

Y = Y.replace('Healthy', 0)
Y = Y.replace('URTI', 1)
Y = Y.replace('Asthma', 1)
Y = Y.replace('COPD', 1)
Y = Y.replace('LRTI', 1)
Y = Y.replace('Bronchiectasis', 1)
Y = Y.replace('Pneumonia', 1)
Y = Y.replace('Bronchiolitis', 1)
Y = Y.replace(np.nan, '0')

Y = pd.DataFrame(Y)
print(dataset)

FileNotFoundError: [Errno 2] No such file or directory: '../input/respiratory-sound-database/demographic_info.txt'

In [ ]:
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

dataset_train = dataset
dataset_train = dataset_train.drop(columns='patient_number')
dataset_train = dataset_train.drop(columns='adult_BMI')
dataset_train = dataset_train.drop(columns='child_weight')
dataset_train = dataset_train.drop(columns='child_height')
print(dataset_train)

In [2]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def plotConfusionMatrix(cf):
    disp = ConfusionMatrixDisplay(confusion_matrix=cf)
    return disp.plot()

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt

train_features, test_features, train_labels, test_labels = train_test_split(dataset_train, np.ravel(Y), test_size = 0.25)
#train_features, val_features, train_labels, val_labels = train_test_split(train_features, np.ravel(train_labels), test_size = 0.25)

max_estimators = np.linspace(1, 30, 30, endpoint=True)

test_errors = []
train_errors = []
#val_errors = []


for i in max_estimators:
    rf = RandomForestClassifier(n_estimators=int(i), random_state = 42, max_depth = 7)# Train the model on training data
    rf.fit(train_features, train_labels);

    train_pred = rf.predict(train_features)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(train_labels, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    error = sum(train_pred != train_labels)/len(train_labels)
    train_errors.append(1-error)
    '''
    val_pred = rf.predict(val_features)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(val_labels, val_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    error = sum(val_pred != val_labels)/len(val_labels)
    val_errors.append(1-error)
    '''
    test_pred = rf.predict(test_features)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(test_labels, test_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    error = sum(test_pred != test_labels)/len(test_labels)
    test_errors.append(1-error)
    
plt.rcParams['figure.figsize'] = [15, 15]
plt.subplot(221)
line1, = plt.plot(max_estimators, train_errors, 'g', label="Train accuracy")
line2, = plt.plot(max_estimators, test_errors, 'b', label="Test accuracy")
#line3, = plt.plot(max_estimators, val_errors, 'r', label="Val accuracy")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Accuracy', size = 20)
plt.xlabel('Forest size', size = 20)
plt.legend(prop={'size': 15})
plt.title('Random forest depth tree analysis', size = 25)
plt.show()
    

NameError: name 'train_test_split' is not defined

In [ ]:
max_depths = np.linspace(1, 32, 32, endpoint=True)
x_train, x_test, y_train, y_test = train_test_split(dataset_children, np.ravel(labels_children), test_size=0.15)

train_results = []
test_results = []
train_errors =[]
test_errors = []

for max_depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(x_train, y_train)
    train_pred = clf.predict(x_train)
    
    #Another way to obtain accuracy and some other interesting metrics
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    error = sum(train_pred != y_train)/len(y_train)
    train_results.append(roc_auc)
    train_errors.append(error)
    
    y_pred = clf.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    error = sum(y_pred != y_test)/len(y_test)
    
    test_errors.append(error)
    test_results.append(roc_auc)

plt.rcParams['figure.figsize'] = [15, 15]
plt.subplot(221)
line1, = plt.plot(max_depths, train_errors, 'b', label="Train error")
line2, = plt.plot(max_depths, test_errors, 'r', label="Test error")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Error')
plt.xlabel('Tree depth')


plt.subplot(222)
line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('Tree depth')


plt.show()